# Preprocessing Dataset

In [102]:
import numpy as np
import pandas as pd

dataset = pd.read_csv("Datasets\dataset_extracted.csv")
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset.set_index('timestamp', inplace=True)
dataset

,id,value,dayofweek,hour
timestamp,,,,
2014-07-01 00:00:00,0,10844,1,0.0
2014-07-01 00:30:00,1,8127,1,0.5
2014-07-01 01:00:00,2,6210,1,1.0
2014-07-01 01:30:00,3,4656,1,1.5
2014-07-01 02:00:00,4,3820,1,2.0
...,...,...,...,...
2015-01-31 21:30:00,10315,24670,5,21.5
2015-01-31 22:00:00,10316,25721,5,22.0
2015-01-31 22:30:00,10317,27309,5,22.5


In [103]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10320 entries, 2014-07-01 00:00:00 to 2015-01-31 23:30:00
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         10320 non-null  int64  
 1   value      10320 non-null  int64  
 2   dayofweek  10320 non-null  int64  
 3   hour       10320 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 403.1 KB


## Feature Engineering

Labeling

In [104]:
dataset['date'] = dataset.index.date

anomaly_dates = [
    '2014-11-02',
    '2014-11-27',
    '2014-12-24',
    '2014-12-25',
    '2015-01-01',
    '2015-01-26',
    '2015-01-27'
]
anomaly_dates = [pd.to_datetime(date).date() for date in anomaly_dates]
dataset['isAnomaly'] = dataset['date'].isin(anomaly_dates)

dataset.loc[dataset['isAnomaly']]

,id,value,dayofweek,hour,date,isAnomaly
timestamp,,,,,,
2014-11-02 00:00:00,5952,25110,6,0.0,2014-11-02,True
2014-11-02 00:30:00,5953,23109,6,0.5,2014-11-02,True
2014-11-02 01:00:00,5954,39197,6,1.0,2014-11-02,True
2014-11-02 01:30:00,5955,35212,6,1.5,2014-11-02,True
2014-11-02 02:00:00,5956,13259,6,2.0,2014-11-02,True
...,...,...,...,...,...,...
2015-01-27 21:30:00,10123,11073,1,21.5,2015-01-27,True
2015-01-27 22:00:00,10124,10559,1,22.0,2015-01-27,True
2015-01-27 22:30:00,10125,9121,1,22.5,2015-01-27,True


Created lag values

In [105]:
col = 'value_lag_0'
dataset[col] = dataset['value']
timer = 23.5
day = 0
for i in range(1, 49):
  dataset[f'value_lag_{i}'] = dataset[f'value_lag_{i-1}'].shift(1)

  # Filter for matching day of week, hour, and non-anomalies
  filtered_data = dataset.loc[(dataset['dayofweek'] == day) & (dataset['hour'] == timer) & (~dataset['isAnomaly'])]

  # Check if there's any data for replacement
  if not filtered_data.empty:
      replace = np.mean(filtered_data['value'])  # Calculate mean of 'value' column
  else:
      replace = np.nan  # Use NaN if no data found

  dataset[f'value_lag_{i}'] = dataset[f'value_lag_{i}'].fillna(replace)
  dataset[f'value_lag_{i}'] = round(dataset[f'value_lag_{i}'])
  dataset[f'value_lag_{i}'] = dataset[f'value_lag_{i}'].astype(int)
  timer -= 0.5

dataset.head()

,id,value,dayofweek,hour,date,isAnomaly,value_lag_0,value_lag_1,value_lag_2,value_lag_3,...,value_lag_39,value_lag_40,value_lag_41,value_lag_42,value_lag_43,value_lag_44,value_lag_45,value_lag_46,value_lag_47,value_lag_48
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-07-01 00:00:00,0,10844,1,0.0,2014-07-01,False,10844,12009,14587,16758,...,2362,2399,2101,2306,2792,3516,4173,5403,7068,9119
2014-07-01 00:30:00,1,8127,1,0.5,2014-07-01,False,8127,10844,12009,14587,...,2733,2362,2399,2101,2306,2792,3516,4173,5403,7068
2014-07-01 01:00:00,2,6210,1,1.0,2014-07-01,False,6210,8127,10844,12009,...,4357,2733,2362,2399,2101,2306,2792,3516,4173,5403
2014-07-01 01:30:00,3,4656,1,1.5,2014-07-01,False,4656,6210,8127,10844,...,6414,4357,2733,2362,2399,2101,2306,2792,3516,4173
2014-07-01 02:00:00,4,3820,1,2.0,2014-07-01,False,3820,4656,6210,8127,...,10365,6414,4357,2733,2362,2399,2101,2306,2792,3516


In [106]:
mapping = {
    0:1,
    1:2,
    2:3,
    3:4,
    4:5,
    5:6,
    6:0
}
dataset['dayofweek'] = dataset['dayofweek'].map(mapping)
dataset['hour'] = - np.sin(2 * np.pi * dataset['hour'] / 24)

dataset.head()

,id,value,dayofweek,hour,date,isAnomaly,value_lag_0,value_lag_1,value_lag_2,value_lag_3,...,value_lag_39,value_lag_40,value_lag_41,value_lag_42,value_lag_43,value_lag_44,value_lag_45,value_lag_46,value_lag_47,value_lag_48
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-07-01 00:00:00,0,10844,2,-0.000000,2014-07-01,False,10844,12009,14587,16758,...,2362,2399,2101,2306,2792,3516,4173,5403,7068,9119
2014-07-01 00:30:00,1,8127,2,-0.130526,2014-07-01,False,8127,10844,12009,14587,...,2733,2362,2399,2101,2306,2792,3516,4173,5403,7068
2014-07-01 01:00:00,2,6210,2,-0.258819,2014-07-01,False,6210,8127,10844,12009,...,4357,2733,2362,2399,2101,2306,2792,3516,4173,5403
2014-07-01 01:30:00,3,4656,2,-0.382683,2014-07-01,False,4656,6210,8127,10844,...,6414,4357,2733,2362,2399,2101,2306,2792,3516,4173
2014-07-01 02:00:00,4,3820,2,-0.500000,2014-07-01,False,3820,4656,6210,8127,...,10365,6414,4357,2733,2362,2399,2101,2306,2792,3516


## Features Scaling

In [107]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def minmax_scaling(columns:list):
    minimum = 0
    maximum = max(dataset['value'])
    for col in columns:
        dataset[col] = (dataset[col] - minimum) / (maximum - minimum)

features = dataset.columns
features = [i for i in features if "value" in i]
minmax_scaling(features)

## Time-Series Decomposition

In [108]:
from statsmodels.tsa.seasonal import seasonal_decompose

decompose_result = seasonal_decompose(dataset['value'], model='additive', period=48)

# Extract components
observed = decompose_result.observed
trend = decompose_result.trend
seasonal = decompose_result.seasonal
residual = decompose_result.resid
dataset['trend'] = trend
dataset['seasonal'] = seasonal
dataset['residual'] = residual

for col in ['trend','residual']:
  val = (dataset['value'] - dataset['seasonal'])/2
  dataset[col].fillna(val,inplace = True)

dataset

C:\Users\weare\AppData\Local\Temp\ipykernel_12624\1640935245.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[col].fillna(val,inplace = True)


,id,value,dayofweek,hour,date,isAnomaly,value_lag_0,value_lag_1,value_lag_2,value_lag_3,...,value_lag_42,value_lag_43,value_lag_44,value_lag_45,value_lag_46,value_lag_47,value_lag_48,trend,seasonal,residual
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-07-01 00:00:00,0,0.276654,2,-0.000000,2014-07-01,False,0.276654,0.306375,0.372146,0.427533,...,0.058831,0.071230,0.089701,0.106462,0.137842,0.180320,0.232645,0.129906,0.016842,0.129906
2014-07-01 00:30:00,1,0.207337,2,-0.130526,2014-07-01,False,0.207337,0.276654,0.306375,0.372146,...,0.053601,0.058831,0.071230,0.089701,0.106462,0.137842,0.180320,0.123814,-0.040290,0.123814
2014-07-01 01:00:00,2,0.158430,2,-0.258819,2014-07-01,False,0.158430,0.207337,0.276654,0.306375,...,0.061204,0.053601,0.058831,0.071230,0.089701,0.106462,0.137842,0.121883,-0.085336,0.121883
2014-07-01 01:30:00,3,0.118785,2,-0.382683,2014-07-01,False,0.118785,0.158430,0.207337,0.276654,...,0.060260,0.061204,0.053601,0.058831,0.071230,0.089701,0.106462,0.124968,-0.131152,0.124968
2014-07-01 02:00:00,4,0.097456,2,-0.500000,2014-07-01,False,0.097456,0.118785,0.158430,0.207337,...,0.069725,0.060260,0.061204,0.053601,0.058831,0.071230,0.089701,0.130517,-0.163577,0.130517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-31 21:30:00,10315,0.629385,6,0.608761,2015-01-31,False,0.629385,0.605123,0.594204,0.637421,...,0.594535,0.657652,0.663316,0.686991,0.685614,0.682170,0.674337,0.233792,0.161801,0.233792
2015-01-31 22:00:00,10316,0.656198,6,0.500000,2015-01-31,False,0.656198,0.629385,0.605123,0.594204,...,0.543868,0.594535,0.657652,0.663316,0.686991,0.685614,0.682170,0.250624,0.154950,0.250624
2015-01-31 22:30:00,10317,0.696711,6,0.382683,2015-01-31,False,0.696711,0.656198,0.629385,0.605123,...,0.485343,0.543868,0.594535,0.657652,0.663316,0.686991,0.685614,0.279331,0.138049,0.279331


## Features Selection

In [109]:
features = dataset.columns
features

remove_cols = [col for col in features if "lag" in col]
add = ['isAnomaly','date']
remove_cols.append('isAnomaly')
remove_cols.append('date')
dataset = dataset[features.drop(remove_cols)]
dataset

,id,value,dayofweek,hour,trend,seasonal,residual
timestamp,,,,,,,
2014-07-01 00:00:00,0,0.276654,2,-0.000000,0.129906,0.016842,0.129906
2014-07-01 00:30:00,1,0.207337,2,-0.130526,0.123814,-0.040290,0.123814
2014-07-01 01:00:00,2,0.158430,2,-0.258819,0.121883,-0.085336,0.121883
2014-07-01 01:30:00,3,0.118785,2,-0.382683,0.124968,-0.131152,0.124968
2014-07-01 02:00:00,4,0.097456,2,-0.500000,0.130517,-0.163577,0.130517
...,...,...,...,...,...,...,...
2015-01-31 21:30:00,10315,0.629385,6,0.608761,0.233792,0.161801,0.233792
2015-01-31 22:00:00,10316,0.656198,6,0.500000,0.250624,0.154950,0.250624
2015-01-31 22:30:00,10317,0.696711,6,0.382683,0.279331,0.138049,0.279331


# Upload preprocessed dataset

In [110]:
features = dataset.columns

dataset[features].to_csv("Datasets\dataset_prep1_tsd.csv")

check = pd.read_csv('Datasets\dataset_prep1_tsd.csv')

check

,timestamp,id,value,dayofweek,hour,trend,seasonal,residual
0,2014-07-01 00:00:00,0,0.276654,2,-0.000000,0.129906,0.016842,0.129906
1,2014-07-01 00:30:00,1,0.207337,2,-0.130526,0.123814,-0.040290,0.123814
2,2014-07-01 01:00:00,2,0.158430,2,-0.258819,0.121883,-0.085336,0.121883
3,2014-07-01 01:30:00,3,0.118785,2,-0.382683,0.124968,-0.131152,0.124968
4,2014-07-01 02:00:00,4,0.097456,2,-0.500000,0.130517,-0.163577,0.130517
...,...,...,...,...,...,...,...,...
10315,2015-01-31 21:30:00,10315,0.629385,6,0.608761,0.233792,0.161801,0.233792
10316,2015-01-31 22:00:00,10316,0.656198,6,0.500000,0.250624,0.154950,0.250624
10317,2015-01-31 22:30:00,10317,0.696711,6,0.382683,0.279331,0.138049,0.279331
10318,2015-01-31 23:00:00,10318,0.678394,6,0.258819,0.285628,0.107137,0.285628


In [111]:
features = features.drop(['residual','trend','seasonal'])

dataset[features].to_csv("Datasets\dataset_prep1.csv")

check = pd.read_csv('Datasets\dataset_prep1.csv')

check

,timestamp,id,value,dayofweek,hour
0,2014-07-01 00:00:00,0,0.276654,2,-0.000000
1,2014-07-01 00:30:00,1,0.207337,2,-0.130526
2,2014-07-01 01:00:00,2,0.158430,2,-0.258819
3,2014-07-01 01:30:00,3,0.118785,2,-0.382683
4,2014-07-01 02:00:00,4,0.097456,2,-0.500000
...,...,...,...,...,...
10315,2015-01-31 21:30:00,10315,0.629385,6,0.608761
10316,2015-01-31 22:00:00,10316,0.656198,6,0.500000
10317,2015-01-31 22:30:00,10317,0.696711,6,0.382683
10318,2015-01-31 23:00:00,10318,0.678394,6,0.258819
